In [2]:
import ollama
import os
import pdfplumber
import pandas as pd
import re
import json
from pathlib import Path
from typing import List, Dict, Optional
import pytesseract
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
import os

In [3]:
import os

# Windows example – adjust the path to where tesseract is installed
os.environ["PATH"] += os.pathsep + r"C:\Program Files\Tesseract-OCR"

# Now test again
import pytesseract
print(pytesseract.get_tesseract_version())

5.5.0.20241111


In [6]:
SELECTED_PDF = '2070_01-01-2025 tot 01-01-2028_akkoord.pdf'

In [7]:
def lees_pdf_ocr(pdf_path: str) -> str:
    """PDF lezen met OCR voor gescande documenten"""
    try:
        import fitz  # PyMuPDF
        from PIL import Image
        import pytesseract
        import io
        
        tekst = ""
        pdf_document = fitz.open(pdf_path)
        print(f"📄 PDF heeft {len(pdf_document)} pagina's (OCR modus)")
        
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            
            # Probeer eerst normale tekst extractie
            page_text = page.get_text()
            
            if page_text.strip():
                # Als er tekst is, gebruik die
                tekst += page_text + "\n\n"
                print(f"   Pagina {page_num + 1}: Tekst gevonden ({len(page_text)} karakters)")
            else:
                # Geen tekst gevonden, gebruik OCR
                print(f"   Pagina {page_num + 1}: Geen tekst - OCR gebruiken...")
                
                # Converteer pagina naar afbeelding
                mat = fitz.Matrix(2, 2)  # 2x zoom voor betere OCR
                pix = page.get_pixmap(matrix=mat)
                img_data = pix.tobytes("png")
                
                # PIL Image voor OCR
                image = Image.open(io.BytesIO(img_data))
                
                # OCR met Tesseract
                ocr_text = pytesseract.image_to_string(image, lang='nld+eng')
                
                if ocr_text.strip():
                    tekst += ocr_text + "\n\n"
                    print(f"   Pagina {page_num + 1}: OCR tekst ({len(ocr_text)} karakters)")
                else:
                    print(f"   Pagina {page_num + 1}: Geen tekst gevonden met OCR")
        
        pdf_document.close()
        print(f"✅ PDF met OCR gelezen: {len(tekst)} totaal karakters")
        return tekst
        
    except ImportError as e:
        print(f"❌ Ontbrekende dependency: {e}")
        print("💡 Installeer: pip install PyMuPDF pillow pytesseract")
        return lees_pdf_advanced(pdf_path)  # Fallback naar normale PDF reader
    except Exception as e:
        print(f"❌ Fout bij OCR PDF extractie: {e}")
        return lees_pdf_advanced(pdf_path)  # Fallback naar normale PDF reader

def lees_pdf_smart(pdf_path: str) -> str:
    """Slimme PDF reader die automatisch kiest tussen normale en OCR extractie"""
    try:
        # Probeer eerst normale extractie
        normale_tekst = lees_pdf(pdf_path)
        
        # Check of er genoeg tekst is gevonden
        if len(normale_tekst.strip()) > 100:  # Als meer dan 100 karakters
            print("📖 Normale PDF extractie succesvol")
            return normale_tekst
        else:
            print("📖 Weinig tekst gevonden - probeer OCR...")
            return lees_pdf_ocr(pdf_path)
            
    except Exception as e:
        print(f"❌ Fout bij slimme PDF extractie: {e}")
        return ""

# Voeg toe aan je bestaande CEL 4 sectie:
print("🔍 OCR PDF functies toegevoegd!")
print("   - lees_pdf_ocr(): Voor gescande PDF's")
print("   - lees_pdf_smart(): Automatische keuze tussen normale/OCR extractie")

🔍 OCR PDF functies toegevoegd!
   - lees_pdf_ocr(): Voor gescande PDF's
   - lees_pdf_smart(): Automatische keuze tussen normale/OCR extractie


In [8]:
tekst = lees_pdf_ocr(SELECTED_PDF)
print(tekst)

📄 PDF heeft 10 pagina's (OCR modus)
   Pagina 1: Tekst gevonden (2097 karakters)
   Pagina 2: Tekst gevonden (1981 karakters)
   Pagina 3: Tekst gevonden (3085 karakters)
   Pagina 4: Tekst gevonden (1982 karakters)
   Pagina 5: Tekst gevonden (2609 karakters)
   Pagina 6: Tekst gevonden (2343 karakters)
   Pagina 7: Tekst gevonden (2274 karakters)
   Pagina 8: Tekst gevonden (1806 karakters)
   Pagina 9: Tekst gevonden (749 karakters)
   Pagina 10: Tekst gevonden (1612 karakters)
✅ PDF met OCR gelezen: 20558 totaal karakters
 
1 
 
Onderhandelingsresultaat CAO Facilitaire ContactCenters 2025-2027 
Privileged & Confidential 
Amersfoort, 14 oktober 2024 
Onderhandelingsresultaat CAO Facilitaire ContactCenters 
1 januari 2025 t/m 31 december 2027 
 
De volgende partijen: 
1. De werkgeversvereniging Facilitaire Contactcenters te Doorn (hierna te noemen: WFC); enerzijds 
en 
2. Vakorganisatie FNV te Utrecht (hierna te noemen: FNV); en 
3. Vakorganisatie CNV te Utrecht (hierna te noemen: CN

In [9]:
# CEL 4: PDF LEES FUNCTIES
print("📖 PDF LEES FUNCTIES DEFINIËREN")
print("=" * 50)

def lees_pdf(pdf_path: str) -> str:
    """Standaard PDF lezen met pdfplumber"""
    try:
        tekst = ""
        with pdfplumber.open(pdf_path) as pdf:
            print(f"📄 PDF heeft {len(pdf.pages)} pagina's")
            
            for i, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                if page_text:
                    tekst += page_text + "\n\n"
                    print(f"   Pagina {i}: {len(page_text)} karakters")
                else:
                    print(f"   Pagina {i}: geen tekst gevonden")
                
        print(f"✅ PDF gelezen: {len(tekst)} totaal karakters")
        return tekst
    except Exception as e:
        print(f"❌ Fout bij lezen PDF: {e}")
        return ""

def lees_pdf_advanced(pdf_path: str) -> str:
    """Geavanceerde PDF lezen met tabellen"""
    try:
        tekst = ""
        with pdfplumber.open(pdf_path) as pdf:
            print(f"📄 PDF heeft {len(pdf.pages)} pagina's")
            
            for i, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                tables = page.extract_tables()
                
                if page_text:
                    tekst += page_text + "\n"
                
                if tables:
                    for table in tables:
                        for row in table:
                            if row:
                                row_text = " | ".join([cell or "" for cell in row])
                                tekst += row_text + "\n"
                    print(f"   Pagina {i}: {len(tables)} tabel(len) gevonden")
                
                tekst += "\n"
                
        print(f"✅ Geavanceerde PDF extractie: {len(tekst)} karakters")
        return tekst
    except Exception as e:
        print(f"❌ Fout bij geavanceerde PDF extractie: {e}")
        return lees_pdf(pdf_path)

print("✅ PDF functies gedefinieerd!")

📖 PDF LEES FUNCTIES DEFINIËREN
✅ PDF functies gedefinieerd!


In [10]:
# CEL 8: REGEX FUNCTIE VOOR LOON ZINNEN
print("🔍 REGEX FUNCTIE VOOR LOON ZINNEN DEFINIËREN")
print("=" * 50)

def vind_loon_zinnen(tekst: str) -> List[str]:
    """Vindt zinnen die mogelijk over loonsverhogingen gaan"""
    
    # Regex patterns voor loonsverhogingen
    percentage_patterns = [
        r'[0-9]+[,.]?[0-9]*\s*%',           # 5%, 3.5%, 2,5%
        r'[0-9]+[,.]?[0-9]*\s*procent',     # 5 procent, 3,5 procent
        r'[0-9]+[,.]?[0-9]*\s*pct',         # 5 pct
    ]
    
    # Loon-gerelateerde woorden
    loon_woorden = [
        'loon', 'salaris', 'bezoldiging', 'vergoeding', 'beloning',
        'verhog', 'toesla', 'indexa', 'aanpass', 'structureel',
        'cao', 'arbeidsvoorwaarden'
    ]
    
    # Split tekst in zinnen
    zinnen = re.split(r'[.!?]+', tekst)
    
    relevante_zinnen = []
    
    for zin in zinnen:
        zin = zin.strip()
        if len(zin) < 20:  # Skip te korte zinnen
            continue
            
        # Check of zin percentage bevat
        heeft_percentage = any(re.search(pattern, zin, re.IGNORECASE) 
                              for pattern in percentage_patterns)
        
        # Check of zin loon-woorden bevat
        heeft_loon_woord = any(woord.lower() in zin.lower() 
                              for woord in loon_woorden)
        
        if heeft_percentage and heeft_loon_woord:
            relevante_zinnen.append(zin)
    
    print(f"✅ {len(relevante_zinnen)} relevante zinnen gevonden")
    return relevante_zinnen

print("✅ Regex functie gedefinieerd!")

🔍 REGEX FUNCTIE VOOR LOON ZINNEN DEFINIËREN
✅ Regex functie gedefinieerd!


In [12]:
# CEL 9: LOON ZINNEN ZOEKEN IN PDF TEKST
print("🔍 LOON ZINNEN ZOEKEN IN PDF TEKST")
print("=" * 50)

if tekst:
    print(f"📄 Doorzoeken van {len(tekst)} karakters tekst...")
    
    # Zoek relevante zinnen
    loon_zinnen = vind_loon_zinnen(tekst)
    
    if loon_zinnen:
        print(f"\n📋 GEVONDEN LOON ZINNEN ({len(loon_zinnen)}):")
        print("=" * 50)
        
        for i, zin in enumerate(loon_zinnen, 1):
            print(f"{i}. {zin}")
            print("-" * 50)
            
        # Bewaar voor volgende stap
        GEVONDEN_ZINNEN = loon_zinnen
        print("✅ Zinnen opgeslagen voor analyse!")
    else:
        print("❌ Geen relevante loon zinnen gevonden")
        print("💡 Probeer de geavanceerde PDF extractie of controleer je PDF inhoud")
        GEVONDEN_ZINNEN = []
else:
    print("❌ Geen PDF tekst beschikbaar")
    GEVONDEN_ZINNEN = []

🔍 LOON ZINNEN ZOEKEN IN PDF TEKST
📄 Doorzoeken van 20558 karakters tekst...
✅ 21 relevante zinnen gevonden

📋 GEVONDEN LOON ZINNEN (21):
1. 2 
 
Onderhandelingsresultaat CAO Facilitaire ContactCenters 2025-2027 
Privileged & Confidential 
 
De afstand van het WML is gedurende de cao per jaar als volgt: 
• 
1 januari 2025: er wordt afstand gehouden van de hoogte van het WML ter grootte van 3,75%
--------------------------------------------------
2. Per jaar betekent dit het volgende: 
• 
Het minimum bruto uursalaris van de Klantadviseur Level 1 per 1 januari 2025 wordt berekend 
door op het wettelijk minimumloon van 1 januari 2025 een percentage van 3,75% te tellen
--------------------------------------------------
3. • 
Het minimum bruto uursalaris van de Klantadviseur Level 1 per 1 januari 2026 wordt berekend 
door op het wettelijk minimumloon van 1 januari 2026 een percentage van 3,85% te tellen
--------------------------------------------------
4. • 
Het minimum bruto uursalaris van

In [16]:
import re
from typing import List, Dict

def extraheer_loonsverhogingen_regex(zinnen: List[str]) -> List[Dict]:
    """
    Verbeterde regex-only functie om loonsverhogingen te extraheren
    """
    
    # Verbeterde regex patronen
    percentage_pattern = r'\b(\d{1,2}(?:[,.]\d+)?)\s*%'
    
    # Uitgebreide Nederlandse datum patronen
    datum_patterns = [
        r'(\d{1,2})\s+(januari|februari|maart|april|mei|juni|juli|augustus|september|oktober|november|december)\s+(\d{4})',
        r'per\s+(\d{1,2})\s+(januari|februari|maart|april|mei|juni|juli|augustus|september|oktober|november|december)\s+(\d{4})',
        r'(\d{1,2})[-/](\d{1,2})[-/](\d{4})',
        r'per\s+(\d{1,2})[-/](\d{1,2})[-/](\d{4})',
        r'(\d{4})[-/](\d{1,2})[-/](\d{1,2})',
        r'in\s+(\d{4})',  # "in 2025", "in 2026"
        r'(\d{4})',  # gewoon jaartal
    ]
    
    # Nederlandse maanden mapping
    maanden = {
        'januari': '01', 'februari': '02', 'maart': '03', 'april': '04',
        'mei': '05', 'juni': '06', 'juli': '07', 'augustus': '08',
        'september': '09', 'oktober': '10', 'november': '11', 'december': '12'
    }
    
    # Strengere uitsluitingen - alleen echte voorbeelden/berekeningen
    skip_termen = ['rekenvoorbeeld', 'voorbeeld:', 'stel dat', 'ter illustratie', 'hypothetisch', 'fictief']
    
    resultaten = []
    
    for zin in zinnen:
        zin_lower = zin.lower()
        
        # Skip alleen echte voorbeelden - niet alles met "voorbeeld"
        if any(term in zin_lower for term in skip_termen):
            # Maar wel toestaan als het concrete percentages bevat voor toekomstige jaren
            if not any(jaar in zin for jaar in ['2025', '2026', '2027']):
                continue
                
        # Vind percentages
        percentages = re.findall(percentage_pattern, zin, re.IGNORECASE)
        
        # Vind datums/jaren
        datums = []
        jaren = []
        
        for pattern in datum_patterns:
            matches = re.findall(pattern, zin, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    if len(match) == 3 and not match[1].isdigit():  # Tekstuele maand
                        dag, maand_naam, jaar = match
                        maand_nr = maanden.get(maand_naam.lower(), '01')
                        datum = f"{jaar}-{maand_nr}-{dag.zfill(2)}"
                        datums.append(datum)
                        jaren.append(jaar)
                    elif len(match) == 3 and match[1].isdigit():  # Numerieke datum
                        if len(match[0]) == 4:  # YYYY-MM-DD
                            jaar, maand, dag = match
                        else:  # DD-MM-YYYY
                            dag, maand, jaar = match
                        datum = f"{jaar}-{maand.zfill(2)}-{dag.zfill(2)}"
                        datums.append(datum)
                        jaren.append(jaar)
                    elif len(match) == 1:  # Alleen jaar
                        jaar = match
                        jaren.append(jaar)
                else:  # Enkele match
                    jaar = match
                    jaren.append(jaar)
        
        # Als we percentages hebben, probeer ze te koppelen
        if percentages:
            # Zoek naar relevante loon-gerelateerde termen
            loon_termen = ['verhoging', 'stijg', 'verhoog', 'cao', 'salaris', 'loon', 'uursalaris', 
                          'percentage', 'afstand', 'minimumloon', 'bijdrage', 'toeslag']
            
            heeft_loon_context = any(term in zin_lower for term in loon_termen)
            
            if heeft_loon_context:
                # Gebruik datums als beschikbaar, anders jaren
                tijdsaanduidingen = datums if datums else []
                
                # Als geen specifieke datums, maar wel jaren gevonden
                if not tijdsaanduidingen and jaren:
                    for jaar in jaren:
                        if jaar in ['2025', '2026', '2027']:  # Alleen toekomstige jaren
                            tijdsaanduidingen.append(f"{jaar}-01-01")  # Default naar 1 januari
                
                # Als nog steeds geen tijdsaanduiding, zoek naar jaartallen in de zin
                if not tijdsaanduidingen:
                    jaar_pattern = r'\b(202[5-9])\b'  # 2025-2029
                    gevonden_jaren = re.findall(jaar_pattern, zin)
                    for jaar in gevonden_jaren:
                        tijdsaanduidingen.append(f"{jaar}-01-01")
                
                # Combineer percentages met tijdsaanduidingen
                for i, percentage in enumerate(percentages):
                    try:
                        perc_value = float(percentage.replace(',', '.'))
                        # Ruimere range voor CAO percentages
                        if 0.1 <= perc_value <= 15:  # Van 0.1% tot 15%
                            if tijdsaanduidingen:
                                # Gebruik cyclisch de tijdsaanduidingen
                                tijdsaanduiding = tijdsaanduidingen[i % len(tijdsaanduidingen)]
                                
                                # Skip historische data
                                if '2024' not in tijdsaanduiding:
                                    resultaat = {
                                        'percentage': f"{percentage}%",
                                        'datum': tijdsaanduiding,
                                        'originele_zin': zin.strip()
                                    }
                                    resultaten.append(resultaat)
                    except ValueError:
                        continue
    
    # Verwijder duplicaten
    unieke_resultaten = []
    gezien = set()
    
    for resultaat in resultaten:
        sleutel = f"{resultaat['datum']}_{resultaat['percentage']}"
        if sleutel not in gezien:
            gezien.add(sleutel)
            unieke_resultaten.append(resultaat)
    
    # Sorteer op datum
    unieke_resultaten.sort(key=lambda x: x['datum'])
    
    return unieke_resultaten

def print_resultaten(resultaten: List[Dict]):
    """
    Print de gevonden loonsverhogingen netjes
    """
    if not resultaten:
        print("❌ Geen loonsverhogingen gevonden")
        return
    
    print(f"\n📋 GEVONDEN LOONSVERHOGINGEN ({len(resultaten)}):")
    print("=" * 60)
    
    for i, resultaat in enumerate(resultaten, 1):
        print(f"{i}. DATUM: {resultaat['datum']}")
        print(f"   PERCENTAGE: {resultaat['percentage']}")
        print(f"   CONTEXT: {resultaat['originele_zin'][:100]}...")
        print("-" * 60)
    
    print(f"\n📈 OVERZICHT:")
    for resultaat in resultaten:
        print(f"• {resultaat['datum']}: {resultaat['percentage']}")

# Hoofduitvoering - gebruik de GEVONDEN_ZINNEN variabele
print("🚀 REGEX EXTRACTIE UITVOEREN")
print("=" * 50)

if 'GEVONDEN_ZINNEN' in globals() and GEVONDEN_ZINNEN:
    print(f"📤 Verwerken van {len(GEVONDEN_ZINNEN)} gevonden zinnen...")
    
    # Voer regex extractie uit
    resultaten = extraheer_loonsverhogingen_regex(GEVONDEN_ZINNEN)
    
    if resultaten:
        print_resultaten(resultaten)
        
        # Bewaar voor verdere analyse
        FINALE_RESULTATEN = resultaten
        print("✅ Regex extractie succesvol voltooid!")
    else:
        print("❌ Geen loonsverhogingen gevonden")
        FINALE_RESULTATEN = []
else:
    print("❌ Geen GEVONDEN_ZINNEN beschikbaar")
    print("💡 Voer eerst de loon zinnen zoekfunctie uit")
    FINALE_RESULTATEN = []

🚀 REGEX EXTRACTIE UITVOEREN
📤 Verwerken van 21 gevonden zinnen...

📋 GEVONDEN LOONSVERHOGINGEN (12):
1. DATUM: 2025-01-01
   PERCENTAGE: 3,75%
   CONTEXT: 2 
 
Onderhandelingsresultaat CAO Facilitaire ContactCenters 2025-2027 
Privileged & Confidential 
 ...
------------------------------------------------------------
2. DATUM: 2025-01-01
   PERCENTAGE: 2%
   CONTEXT: Levels 1 t/m 3 
In 2025, 2026 en 2027 zal het minimum basissalaris van de Levels 2 en 3 respectievel...
------------------------------------------------------------
3. DATUM: 2025-01-01
   PERCENTAGE: 4%
   CONTEXT: Levels 1 t/m 3 
In 2025, 2026 en 2027 zal het minimum basissalaris van de Levels 2 en 3 respectievel...
------------------------------------------------------------
4. DATUM: 2025-01-01
   PERCENTAGE: 2,75%
   CONTEXT: Rekenvoorbeeld 2025 
Op 1 januari 2025 gaat het wettelijk minimumloon omhoog met 2,75%...
------------------------------------------------------------
5. DATUM: 2025-01-01
   PERCENTAGE: 3,88%
 